<img src="https://pandas.pydata.org/_static/pandas_logo.png" width="400" align="left"/>

* [PANDAS: 3. Pivot Table](#1)
* [PANDAS: 4. Group By](#2)

In [1]:
from IPython.display import IFrame

<a id='1'></a>

# PANDAS: 3. Pivot Table

In [2]:
IFrame(src='https://www.youtube.com/embed/ODFpGo7UomA', width=640, height=400)

In [3]:
import pandas as pd

In [4]:
!head -12 data/env_wastrt.tsv

unit,hazard,wst_oper,waste,geo\time	2016 	2014 	2012 	2010 	2008 	2006 	2004 
KG_HAB,HAZ,DSP_I,DSP_OTH,IS	: 	: b	: 	: 	: 	: 	: 
KG_HAB,HAZ,DSP_I,INC_OTH,AT	: 	: 	: 	: 	: c	2 	2 
KG_HAB,HAZ,DSP_I,INC_OTH,BE	: 	: 	: 	: 	2 	0 	: 
KG_HAB,HAZ,DSP_I,INC_OTH,BG	: 	: 	: 	: 	0 	: 	0 
KG_HAB,HAZ,DSP_I,INC_OTH,CY	: 	: 	: 	: 	0 	0 	: 
KG_HAB,HAZ,DSP_I,INC_OTH,CZ	: 	: 	: 	: 	0 	0 	: c
KG_HAB,HAZ,DSP_I,INC_OTH,DE	: 	: 	: 	: 	2 	2 	1 
KG_HAB,HAZ,DSP_I,INC_OTH,DK	: 	: 	: 	: 	0 	: 	: 
KG_HAB,HAZ,DSP_I,INC_OTH,EE	: 	: 	: 	: 	0 	0 	0 
KG_HAB,HAZ,DSP_I,INC_OTH,EL	: 	: 	: 	: 	0 	: 	0 
KG_HAB,HAZ,DSP_I,INC_OTH,ES	: 	: 	: 	: 	0 	1 	0 


In [5]:
df = pd.read_table('data/env_wastrt.tsv',header=0,sep=',|\t',engine='python',
                  names=['unit','hazard','operation','waste','country',2016,2014,2012,
                        2010,2008,2006,2004])

In [6]:
df.loc[df.waste=='W074','waste']='Plastic'
df.loc[df.waste=='W071','waste']='Glass'
df.loc[df.waste=='W06_07A','waste']='Recyclable'
df.loc[df.waste=='W12-13','waste']='Mineral'
df.loc[df.waste=='W10','waste']='Mixed ordinary wastes'
df.loc[df.waste=='W073','waste']='Rubber'
df.loc[df.waste=='W072','waste']='Paper and cardboard'
df.loc[df.waste=='W081','waste']='Vehicles'
df.loc[df.waste=='W01-05','waste']='Chemical&Medical'
df.loc[df.waste=='W075','waste']='Wood'
df.loc[df.waste=='W076','waste']='Textile'
df.loc[df.waste=='W063','waste']='Metal'

In [7]:
df.loc[df.operation=='DSP_L','operation']='Landfill'
df.loc[df.operation=='DSP_I','operation']='Incineration'
df.loc[df.operation=='RCV_R','operation']='Recycling'
df.loc[df.operation=='TRT','operation']='Waste treatment'
df.loc[df.operation=='DSP_L_OTH','operation']='Landfill&Other'
df.loc[df.operation=='DSP_OTH','operation']='Other'
df.loc[df.operation=='RCV_E','operation']='Energy recovery'
df.loc[df.operation=='RCV_R_B','operation']='Recycling&Backfilling'
df.loc[df.operation=='RCV_B','operation']='Backfilling'

In [8]:
df.iloc[:,5:] = df.iloc[:,5:].applymap(lambda x: x.rstrip('bcdefinprsuz')) 
df.iloc[:,5:] = df.iloc[:,5:].apply(pd.to_numeric,errors='coerce')

In [9]:
df.head()

,unit,hazard,operation,waste,country,2016,2014,2012,2010,2008,2006,2004
0,KG_HAB,HAZ,Incineration,DSP_OTH,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KG_HAB,HAZ,Incineration,INC_OTH,AT,NaN,NaN,NaN,NaN,NaN,2,2
2,KG_HAB,HAZ,Incineration,INC_OTH,BE,NaN,NaN,NaN,NaN,2,0,NaN
3,KG_HAB,HAZ,Incineration,INC_OTH,BG,NaN,NaN,NaN,NaN,0,NaN,0
4,KG_HAB,HAZ,Incineration,INC_OTH,CY,NaN,NaN,NaN,NaN,0,0,NaN


In [10]:
df[(df.unit=='KG_HAB')&(df.waste=='TOTAL')&(df.hazard=='NHAZ')].head()

,unit,hazard,operation,waste,country,2016,2014,2012,2010,2008,2006,2004
28967,KG_HAB,NHAZ,Incineration,TOTAL,AL,NaN,13,NaN,NaN,NaN,NaN,NaN
28968,KG_HAB,NHAZ,Incineration,TOTAL,AT,NaN,2,1,189,182,159,209
28969,KG_HAB,NHAZ,Incineration,TOTAL,BE,136,125,228,167,348,210,228
28970,KG_HAB,NHAZ,Incineration,TOTAL,BG,1,2,1,0,1,16,13
28971,KG_HAB,NHAZ,Incineration,TOTAL,CY,0,0,8,8,18,19,9


In [11]:
df.to_csv('data/wastrt.csv')

# Pivot Table

In [12]:
(df[(df.unit=='KG_HAB')&(df.waste=='TOTAL')&(df.hazard=='NHAZ')]
  .pivot_table(index='country',columns='operation',values=2014,aggfunc='first')
  ).loc[['SE','IT','DK','EL']]

operation,Backfilling,Energy recovery,Incineration,Landfill,Landfill&Other,Other,Recycling,Recycling&Backfilling,Waste treatment
country,,,,,,,,,
SE,261.0,770.0,0.0,8773.0,14173.0,5400.0,1523.0,1784.0,16728.0
IT,5.0,41.0,103.0,319.0,319.0,0.0,1605.0,1610.0,2074.0
DK,0.0,595.0,0.0,456.0,471.0,16.0,1778.0,1778.0,2844.0
EL,499.0,12.0,2.0,5293.0,5441.0,148.0,191.0,690.0,6145.0


In [13]:
(df[(df.unit=='KG_HAB')&(df.waste=='Mineral')&(df.hazard=='NHAZ')]
  .pivot_table(index='country',columns='operation',values=2014,aggfunc='first')
  ).loc[['SE','IT','DK','EL']]

operation,Backfilling,Energy recovery,Incineration,Landfill,Landfill&Other,Other,Recycling,Recycling&Backfilling,Waste treatment
country,,,,,,,,,
SE,261.0,2.0,0.0,8712.0,14112.0,5400.0,929.0,1190.0,15304.0
IT,5.0,0.0,0.0,109.0,109.0,0.0,778.0,783.0,893.0
DK,0.0,26.0,0.0,436.0,436.0,0.0,1358.0,1358.0,1819.0
EL,499.0,0.0,0.0,4863.0,5008.0,145.0,33.0,532.0,5539.0


In [14]:
(df[(df.unit=='KG_HAB')&(df.waste=='Plastic')&(df.hazard=='NHAZ')]
  .pivot_table(index='country',columns='operation',values=2014,aggfunc='first')
  ).loc[['SE','IT','DK','EL']]

operation,Backfilling,Energy recovery,Incineration,Landfill,Landfill&Other,Other,Recycling,Recycling&Backfilling,Waste treatment
country,,,,,,,,,
SE,0.0,7.0,0.0,0.0,0.0,0.0,8.0,8.0,16.0
IT,0.0,1.0,0.0,1.0,1.0,0.0,34.0,34.0,37.0
DK,0.0,1.0,0.0,0.0,0.0,0.0,11.0,11.0,12.0
EL,0.0,0.0,0.0,6.0,6.0,0.0,3.0,3.0,10.0


In [15]:
(df[(df.unit=='KG_HAB')&(df.waste=='Rubber')&(df.hazard=='NHAZ')]
  .pivot_table(index='country',columns='operation',values=2014,aggfunc='first')
  ).loc[['SE','IT','DK','EL']]

operation,Backfilling,Energy recovery,Incineration,Landfill,Landfill&Other,Other,Recycling,Recycling&Backfilling,Waste treatment
country,,,,,,,,,
SE,0.0,3.0,0.0,0.0,0.0,0.0,5.0,5.0,8.0
IT,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0
DK,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,10.0
EL,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [16]:
(df[(df.unit=='KG_HAB')&((df.waste=='Plastic')|(df.waste=='Rubber'))&(df.hazard=='NHAZ')]
  .pivot_table(index='country',columns='operation',values=2014,aggfunc='sum')
  ).loc[['SE','IT','DK','EL']]

operation,Backfilling,Energy recovery,Incineration,Landfill,Landfill&Other,Other,Recycling,Recycling&Backfilling,Waste treatment
country,,,,,,,,,
SE,0.0,10.0,0.0,0.0,0.0,0.0,13.0,13.0,24.0
IT,0.0,2.0,0.0,1.0,1.0,0.0,36.0,36.0,40.0
DK,0.0,1.0,0.0,0.0,0.0,0.0,21.0,21.0,22.0
EL,0.0,0.0,0.0,6.0,6.0,0.0,4.0,4.0,11.0


<a id='2'></a>

# PANDAS: 4. Group By

In [17]:
IFrame(src='https://www.youtube.com/embed/oh8UijClQoE', width=640, height=400)

# MultiIndex Creation (see previous lecture)

In [18]:
df.set_index(keys=['unit','hazard','waste','operation','country'],inplace=True)
df.head()

2016 2014 2012 2010 2008 2006 2004
unit   hazard waste   operation    country                                   
KG_HAB HAZ    DSP_OTH Incineration IS       NaN  NaN  NaN  NaN  NaN  NaN  NaN
              INC_OTH Incineration AT       NaN  NaN  NaN  NaN  NaN    2    2
                                   BE       NaN  NaN  NaN  NaN    2    0  NaN
                                   BG       NaN  NaN  NaN  NaN    0  NaN    0
                                   CY       NaN  NaN  NaN  NaN    0    0  NaN

In [19]:
df.xs(['KG_HAB','NHAZ','Plastic'],level=[0,1,2]).head()

2016 2014 2012 2010 2008 2006 2004
operation    country                                   
Incineration AT       NaN  NaN  NaN    0  NaN  NaN  NaN
             BE         2    1    0    0  NaN  NaN  NaN
             BG         0    0    0    0  NaN  NaN  NaN
             CY         0    0    0    0  NaN  NaN  NaN
             CZ         0    0  NaN    0  NaN  NaN  NaN

# Unstack

In [20]:
df.xs(['KG_HAB','NHAZ','Plastic'],level=[0,1,2]).unstack(0).head()

2016                                                       \
operation Backfilling Energy recovery Incineration Landfill Landfill&Other   
country                                                                      
AT                  0             NaN          NaN        0              0   
BE                  0               1            2        3              3   
BG                  0               2            0        0              0   
CY                  0               0            0        0              0   
CZ                  7               6            0        5              5   

                                                                       2014  \
operation Other Recycling Recycling&Backfilling Waste treatment Backfilling   
country                                                                       
AT            0        32                    32              37           0   
BE            0        23                    23              28           0   
BG            0        30                    30              32         NaN   
CY            0         0                     0               0           0   
CZ            0        18                    24              35           4   

                ...                  2006        2004                  \
operation       ...       Waste treatment Backfilling Energy recovery   
country         ...                                                     
AT              ...                    20         NaN             NaN   
BE              ...                    12         NaN             NaN   
BG              ...                     3         NaN             NaN   
CY              ...                     5         NaN             NaN   
CZ              ...                     9         NaN             NaN   

                                                                \
operation Incineration Landfill Landfill&Other Other Recycling   
country                                                          
AT                 NaN      NaN            NaN   NaN       NaN   
BE                 NaN      NaN            NaN   NaN       NaN   
BG                 NaN      NaN            NaN   NaN       NaN   
CY                 NaN      NaN            NaN   NaN       NaN   
CZ                 NaN      NaN            NaN   NaN       NaN   

                                                 
operation Recycling&Backfilling Waste treatment  
country                                          
AT                           43              43  
BE                          NaN              30  
BG                            1               1  
CY                           14              14  
CZ                            6               6  

[5 rows x 63 columns]

In [21]:
df.xs(['KG_HAB','NHAZ','Plastic'],level=[0,1,2]).unstack(0).loc[['SE','IT','DK','EL'],2014]

operation,Backfilling,Energy recovery,Incineration,Landfill,Landfill&Other,Other,Recycling,Recycling&Backfilling,Waste treatment
country,,,,,,,,,
SE,0,7,0,0,0,0,8,8,16
IT,0,1,0,1,1,0,34,34,37
DK,0,1,0,0,0,0,11,11,12
EL,0,0,0,6,6,0,3,3,10


In [22]:
( df.xs(['KG_HAB','NHAZ','Plastic'],level=[0,1,2])
 .loc[(['Energy recovery','Landfill','Recycling','Waste treatment']
       ,['SE','IT','DK','EL']),(2014,2010)]
).unstack(0)

2014                                               2010  \
operation Landfill Energy recovery Recycling Waste treatment Landfill   
country                                                                 
DK               0               1        11              12        0   
EL               6               0         3              10       12   
IT               1               1        34              37        1   
SE               0               7         8              16        0   

                                                     
operation Energy recovery Recycling Waste treatment  
country                                              
DK                      0        12              13  
EL                      0         2              14  
IT                      0        28              30  
SE                      9         5              14

# GroupBy

In [23]:
for name, group in df.xs(['T','NHAZ','Plastic'],level=[0,1,2])[2014].groupby('country'):
    print(name,type(group),group)

AL <class 'pandas.core.series.Series'> operation        country
Waste treatment  AL         0
Name: 2014, dtype: object
AT <class 'pandas.core.series.Series'> operation              country
Incineration           AT            NaN
Landfill               AT              0
Landfill&Other         AT              0
Other                  AT              0
Backfilling            AT              0
Energy recovery        AT            NaN
Recycling              AT         187436
Recycling&Backfilling  AT         187436
Waste treatment        AT         230008
Name: 2014, dtype: object
BE <class 'pandas.core.series.Series'> operation              country
Incineration           BE          16136
Landfill               BE          22120
Landfill&Other         BE          22120
Other                  BE              0
Backfilling            BE              0
Energy recovery        BE          11624
Recycling              BE         204032
Recycling&Backfilling  BE         204032
Waste treatment  

In [24]:
( df.xs(['T','NHAZ','Plastic'],level=[0,1,2]).groupby('country')
 .transform(lambda x : x / x[-1] * 100) )

ZeroDivisionError: ('float division by zero', 'occurred at index 2014')

In [25]:
import numpy as np

In [26]:
( df.xs(['T','NHAZ','Plastic'],level=[0,1,2]).groupby('country')
 .transform(lambda x : x / x[-1] * 100 if x[-1]>0 else x*np.nan) 
 .loc[(['Energy recovery','Landfill','Recycling'],['SE','IT','DK','EL','EU28']),
     ([2014,2010])] ).unstack(0)

2014                                2010                  \
operation   Landfill Energy recovery Recycling  Landfill Energy recovery   
country                                                                    
DK           2.31318          5.3689   92.3179   3.29567         1.76776   
EL           63.7675        0.729383   35.5032   82.5777               0   
EU28         6.04982         15.3915   77.4911   10.2062         15.6701   
IT           1.93994         3.91547   92.8216   4.14758        0.676479   
SE         0.0137497           46.76   53.0757  0.311207         64.9379   

                     
operation Recycling  
country              
DK          94.9366  
EL          17.4223  
EU28        72.4742  
IT          93.8621  
SE          34.7509

# Recalculating the Aggregate EU28
https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes

In [27]:
(df.xs(['T','NHAZ','Plastic'],level=[0,1,2])
 .loc[(['Energy recovery','Landfill','Recycling','Waste treatment'],['EU28']),
     ([2014,2010])]).sort_index().T

operation,Energy recovery,Landfill,Recycling,Waste treatment
country,EU28,EU28,EU28,EU28
2014,1.73e+06,680000,8.71e+06,1.124e+07
2010,1.52e+06,990000,7.03e+06,9.7e+06


In [28]:
(df.xs(['T','NHAZ','Plastic'],level=[0,1,2])
 .drop(labels=['EU28','IS','LI','NO','CH','ME','MK','AL','RS','TR','XK','HA'],level=1)
 .groupby('operation').sum()
 .loc[['Energy recovery','Landfill','Recycling','Waste treatment'],[2014,2010]]).T

operation,Energy recovery,Landfill,Recycling,Waste treatment
2014,1682213.0,680483.0,8683088.0,11242992.0
2010,1470385.0,987780.0,7030405.0,9701209.0


In [29]:
(df.xs(['T','NHAZ','Plastic'],level=[0,1,2])
 .drop(labels=['EU28','IS','LI','NO','CH','ME','MK','AL','RS','TR','XK','HA'],level=1)
 .groupby('operation')
 .agg( lambda x : x.mean() if x.count()>0 else np.nan )
 .loc[['Energy recovery','Landfill','Recycling','Waste treatment'],[2014,2010]]).T

operation,Energy recovery,Landfill,Recycling,Waste treatment
2014,64700.500000,24302.964286,321595.851852,401535.428571
2010,54458.703704,35277.857143,251085.892857,346471.750000


In [30]:
f_mean = lambda x : x.mean() if x.count()>0 else np.nan
f_mean.__name__ = 'mean'
f_std = lambda x : np.std(x) if x.count()>0 else np.nan
f_std.__name__ = 'standard deviation'

(df.xs(['T','NHAZ','Plastic'],level=[0,1,2])
 .drop(labels=['EU28','IS','LI','NO','CH','ME','MK','AL','RS','TR','XK','HA'],level=1)
 .groupby('operation')
 .agg( { 2014 : [f_mean,f_std], 2010 : [f_mean,f_std] } ) ).T

operation                Backfilling  Energy recovery  Incineration  \
2014 mean                1598.576923     64700.500000   3165.851852   
     standard deviation  7991.484729    176323.513987   7253.816700   
2010 mean                   0.000000     54458.703704   4260.714286   
     standard deviation     0.000000    149979.671491   9721.797625   

operation                    Landfill  Landfill&Other       Other  \
2014 mean                24302.964286    25181.333333    0.000000   
     standard deviation  50130.970498    50838.859210    0.000000   
2010 mean                35277.857143    35431.928571  154.071429   
     standard deviation  69192.006370    69129.917756  800.578627   

operation                    Recycling  Recycling&Backfilling  Waste treatment  
2014 mean                321595.851852          324378.730769    401535.428571  
     standard deviation  517927.899710          528460.742516    614415.675541  
2010 mean                251085.892857          276375.080000    346471.750000  
     standard deviation  445222.826384          464940.585347    539147.684052

# Merge

In [31]:
trt_2014 = (df.xs(['T','NHAZ','Plastic'],level=[0,1,2])
 .loc[(['Energy recovery','Landfill','Recycling','Waste treatment'],['SE','IT','DK','EL']),2014]
).unstack(0)
trt_2014

operation,Landfill,Energy recovery,Recycling,Waste treatment
country,,,,
DK,1514,3514,60423,65451
EL,68455,783,38113,107351
IT,43391,87578,2.07616e+06,2.23672e+06
SE,21,71417,81063,152731


In [32]:
df = pd.read_table('data/waste.tsv',header=0,sep=',|\t',engine='python',
                  names=['unit','hazard','activity','waste','country',2016,2014,2012,
                        2010,2008,2006,2004])
df.loc[df.waste=='W074','waste']='Plastic'
df.loc[df.activity=='G-U_X_G4677','activity']='Services'
df.loc[df.activity=='C','activity']='Manufacturing'
df.loc[df.activity=='F','activity']='Construction'
df.loc[df.activity=='EP_HH','activity']='Households'
df.loc[df.activity=='TOTAL_HH','activity']='Total'
df.iloc[:,5:] = df.iloc[:,5:].applymap(lambda x: x.rstrip('bcdefinprsuz')) 
df.iloc[:,5:] = df.iloc[:,5:].apply(pd.to_numeric,errors='coerce')
df = df.set_index(keys=['unit','hazard','waste','activity','country'],inplace=False)
df.head()

2016 2014 2012 2010 2008 2006 2004
unit   hazard waste activity country                                   
KG_HAB HAZ    PRIM  A        AT       NaN    1    1    9  NaN  NaN  NaN
                             BE         2    2    2    2  NaN  NaN  NaN
                             BG         0    0    0    0  NaN  NaN  NaN
                             CY         1    1    1    1  NaN  NaN  NaN
                             CZ         1    0    0    1  NaN  NaN  NaN

In [33]:
gen_2014 = (df.xs(['T','NHAZ','Plastic'],level=[0,1,2])
 .loc[(['Manufacturing','Construction','Households','Total'],['SE','IT','EL','EU28']),2014]
).unstack(0)
gen_2014

activity,Manufacturing,Households,Construction,Total
country,,,,
EL,4798,0,29575,192248
EU28,4.73e+06,2.74e+06,920000,1.735e+07
IT,716131,1.01254e+06,28232,3.21597e+06
SE,46085,96035,500,301187


In [34]:
pd.merge(trt_2014,gen_2014,how='outer',left_index=True,right_index=True)

,Landfill,Energy recovery,Recycling,Waste treatment,Manufacturing,Households,Construction,Total
country,,,,,,,,
DK,1514,3514,60423,65451,NaN,NaN,NaN,NaN
EL,68455,783,38113,107351,4798,0,29575,192248
EU28,NaN,NaN,NaN,NaN,4.73e+06,2.74e+06,920000,1.735e+07
IT,43391,87578,2.07616e+06,2.23672e+06,716131,1.01254e+06,28232,3.21597e+06
SE,21,71417,81063,152731,46085,96035,500,301187


In [35]:
pd.merge(trt_2014,gen_2014,how='inner',left_index=True,right_index=True)

,Landfill,Energy recovery,Recycling,Waste treatment,Manufacturing,Households,Construction,Total
country,,,,,,,,
EL,68455,783,38113,107351,4798,0,29575,192248
IT,43391,87578,2.07616e+06,2.23672e+06,716131,1.01254e+06,28232,3.21597e+06
SE,21,71417,81063,152731,46085,96035,500,301187


In [36]:
pd.merge(trt_2014,gen_2014,how='left',left_index=True,right_index=True)

,Landfill,Energy recovery,Recycling,Waste treatment,Manufacturing,Households,Construction,Total
country,,,,,,,,
DK,1514,3514,60423,65451,NaN,NaN,NaN,NaN
EL,68455,783,38113,107351,4798,0,29575,192248
IT,43391,87578,2.07616e+06,2.23672e+06,716131,1.01254e+06,28232,3.21597e+06
SE,21,71417,81063,152731,46085,96035,500,301187


In [37]:
pd.merge(trt_2014,gen_2014,how='right',left_index=True,right_index=True)

,Landfill,Energy recovery,Recycling,Waste treatment,Manufacturing,Households,Construction,Total
country,,,,,,,,
EL,68455,783,38113,107351,4798,0,29575,192248
EU28,NaN,NaN,NaN,NaN,4.73e+06,2.74e+06,920000,1.735e+07
IT,43391,87578,2.07616e+06,2.23672e+06,716131,1.01254e+06,28232,3.21597e+06
SE,21,71417,81063,152731,46085,96035,500,301187
